In [1]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path(".").absolute().parent))
sys.path.append(str(Path(".").absolute().parent) + "/src/pipeline/pipeline_utils")
import pipeline_utils 
import argparse
import datetime
import pytz
from mlflow_image import decryption 

In [2]:
print(datetime.datetime.now(pytz.timezone("US/Central")).strftime("%Y-%m-%d %H:%M:%S"))


2024-12-03 08:01:32


In [3]:
try:
    args = pipeline_utils.MarkdownArgs().get_args()
except:
    parser = argparse.ArgumentParser()
    parser.add_argument("--COMMIT_ID", required=True, type=str)
    parser.add_argument("--BRANCH", required=True, type=str)
    parser.add_argument("--is_prod", required=False, type=lambda x: (str(x).lower() == 'true'))
    sys.args = [
        "--COMMIT_ID", "1234",
        "--BRANCH", "dev",
        "--is_prod", "False",
    ]
    args = parser.parse_args(sys.args)

usage: ipykernel_launcher.py [-h] --COMMIT_ID COMMIT_ID --BRANCH BRANCH
                             [--is_prod IS_PROD]
ipykernel_launcher.py: error: the following arguments are required: --COMMIT_ID, --BRANCH


In [4]:

PARAMS = pipeline_utils.YamlImport("settings.yml").yaml_import()


BRANCH_ID = args.BRANCH
is_prod = args.is_prod

In [5]:
if BRANCH_ID == "stage" and is_prod == True:
    BRANCH_ID = "prod"
    
ENV = BRANCH_ID
ENV


'dev'

In [6]:
PARAM = PARAMS["envs"][ENV]
PROJECT_ID = PARAM["PROJECT_ID"]
BASE_IMAGE = PARAM["BASE_IMAGE"]
MLFLOW_IMAGE = PARAM["MLFLOW_IMAGE"]

PIPELINE_ROOT = PARAM["PIPELINE_ROOT"]

decryption_obj = decryption.AccessSecret()
ELEMENT_PROJECT_ID = decryption_obj.access_secret(PROJECT_ID, "ELEMENT_PROJECT_ID", "latest")
ELEMENT_PROJECT_TOKEN = decryption_obj.access_secret(PROJECT_ID, "ELEMENT_PROJECT_TOKEN", "latest")
ELEMENT_DECRYPTION_KEY = decryption_obj.access_secret(PROJECT_ID, "ELEMENT_DECRYPTION_KEY", "latest")
ELEMENT_DECRYPTED_TOKEN = decryption_obj.decrypt(ELEMENT_DECRYPTION_KEY, ELEMENT_PROJECT_TOKEN)
# ELEMENT_TOKEN_DECRYPTED = decryption_obj.access_secret(PROJECT_ID, "ELEMENT_TOKEN_DECRYPTED", "latest")

print(f"""
PROJECT_ID: {PROJECT_ID}, 
\nBASE_IMAGE: {BASE_IMAGE},
\nMLFLOW_IMAGE: {MLFLOW_IMAGE},
\nPIPELINE_ROOT: {PIPELINE_ROOT}, 
""")

ImportError: cannot import name 'secretmanager' from 'google.cloud' (unknown location)

In [7]:
MLFLOW_IMAGE

'gcr.io/wmt-mlp-p-price-npd-pricing/markdown-mlflow-dev:latest'

In [8]:
# %%capture
!DOCKER_BUILDKIT=1 docker build --no-cache --progress=plain -t {MLFLOW_IMAGE} . -f ./mlflow_image/Dockerfile \
--build-arg VAR_ELEMENT_PROJECT_ID={ELEMENT_PROJECT_ID} \
--build-arg VAR_ELEMENT_PROJECT_TOKEN={ELEMENT_DECRYPTED_TOKEN} \
--build-arg VAR_ELEMENT_DECRYPTION_KEY={ELEMENT_DECRYPTION_KEY}



invalid argument "{MLFLOW_IMAGE}" for "-t, --tag" flag: invalid reference format: repository name must be lowercase
See 'docker build --help'.


In [9]:
!docker push {MLFLOW_IMAGE}


The push refers to repository [gcr.io/wmt-mlp-p-price-npd-pricing/markdown-mlflow-dev]
An image does not exist locally with the tag: gcr.io/wmt-mlp-p-price-npd-pricing/markdown-mlflow-dev


## to maintain the boot disk without memory leak. 


In [10]:
!docker image prune -f


Total reclaimed space: 0B
